In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from keras import layers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_curve, auc

2024-04-17 10:49:52.113934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 10:49:52.523300: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 10:49:52.525828: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 10:49:54.387506: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [3]:
train_ds = train.flow_from_directory('/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
                                     target_size = (224,224), batch_size = 10)

test_ds = test.flow_from_directory('/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/test',
                                  target_size = (224,224), batch_size = 10)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [4]:
# Set up your data generator using flow_from_directory
train_generator = train.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary')  # Assuming you're doing multi-class classification

# Grab the class labels
train_label = list(train_generator.class_indices.keys())
print("Class Labels:", train_label)

Found 251 images belonging to 3 classes.
Class Labels: ['Covid', 'Normal', 'Viral Pneumonia']


In [5]:
# Set up your data generator using flow_from_directory
test_generator = test.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/test',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary')  # Assuming you're doing multi-class classification

# Grab the class labels
test_label = list(test_generator.class_indices.keys())
print("Class Labels:", test_label)

Found 66 images belonging to 3 classes.
Class Labels: ['Covid', 'Normal', 'Viral Pneumonia']


# MobileNet

# VGG19